In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
df1 = pd.read_csv('./data/pred_val_bert.csv')
df1.columns = ['review_id', 'bert1']
df1['bert2'] = pd.read_csv('./data/pred_val_bert_2.csv')['stars']
df1['bert3'] = pd.read_csv('./data/pred_val_bert_3.csv')['stars']
df1['rnn'] = pd.read_csv('./data/valid_pred.csv')['stars']
df1['lgb'] = pd.read_csv('./data/lgb_val.csv')['predicted_star']
df1.head()

,review_id,bert1,bert2,bert3,rnn,lgb
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,4,4.360081
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,3.824301
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,5,2.929254
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.088829
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,2,2.831013


In [9]:
df1['cat'] = 'valid'

In [10]:
df2 = pd.read_csv('./data/pred_bert.csv')
df2.columns = ['review_id', 'bert1']
df2['bert2'] = pd.read_csv('./data/pred_bert_2.csv')['stars']
df2['bert3'] = pd.read_csv('./data/pred_bert_3.csv')['stars']
df2['rnn'] = pd.read_csv('./data/test_pred.csv')['stars']
df2['lgb'] = pd.read_csv('./data/lgb_pred.csv')['predicted_star']
df2.head()

,review_id,bert1,bert2,bert3,rnn,lgb
0,ewaMkuxfjfCX3ndtk0oRTw,3,4,4,3,3.743851
1,SzkuvIpf0lcVaDgk5x2ipg,4,5,4,5,4.721213
2,ZGg6emL3sqhyyqHD_DJEag,5,5,5,5,4.268382
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.259250
4,dao7SEXhBuMk7TieM4gEZA,5,5,5,5,4.147467


In [11]:
df2['cat'] = 'test'

In [12]:
df = pd.concat([df1, df2])
df['cat'].value_counts()

test     2000
valid    2000
Name: cat, dtype: int64

In [15]:
def round_star(star):
    if star < 1:
        return 1
    elif star > 5:
        return 5
    else:
        return round(star)

In [16]:
df.columns

Index(['review_id', 'bert1', 'bert2', 'bert3', 'rnn', 'lgb', 'cat'], dtype='object')

In [19]:
import numpy as np
df['all_mean'] = df.apply(
    lambda x: np.array([x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb']]).mean(), axis=1)
df['all_mean'] = df['all_mean'].apply(lambda x: round_star(x))
df.head()

,review_id,bert1,bert2,bert3,rnn,lgb,cat,all_mean
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,4,4.360081,valid,3
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,3.824301,valid,5
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,5,2.929254,valid,4
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.088829,valid,5
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,2,2.831013,valid,3


In [21]:
va = pd.read_csv('./data/valid.csv')
y_val = va['stars']

In [34]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def eva(method):
    y_val_pre = df[df.cat == 'valid'][method]
    acc = accuracy_score(y_val, y_val_pre)
    p, r, f1, _ = precision_recall_fscore_support(y_val, y_val_pre, average="macro")
    print("accuracy:", acc, "\tprecision:", p, "\trecall:", r, "\tf1:", f1)
    
eva('all_mean')

accuracy: 0.679 	precision: 0.6158562692864693 	recall: 0.5986992496273191 	f1: 0.6054109712017525


In [ ]:
eva('bert1'), eva('bert2'), eva('bert3'), eva('rnn'), eva('lgb_round')

In [35]:
def normalized_mean(lst):
    lst.sort()
    lst = lst[1:-1]
    return np.array(lst).mean()

df['norm_mean'] = df.apply(
    lambda x: normalized_mean([x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb']]), axis=1)
df['norm_mean'] = df['all_mean'].apply(lambda x: round_star(x))
df.head()

,review_id,bert1,bert2,bert3,rnn,lgb,cat,all_mean,norm_mean
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,4,4.360081,valid,3,3
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,3.824301,valid,5,5
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,5,2.929254,valid,4,4
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.088829,valid,5,5
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,2,2.831013,valid,3,3


In [38]:
eva('norm_mean')

accuracy: 0.679 	precision: 0.6158562692864693 	recall: 0.5986992496273191 	f1: 0.6054109712017525


In [39]:
df['lgb_round'] = df['lgb'].apply(lambda x: round_star(x))
eva('bert1'), eva('bert2'), eva('bert3'), eva('rnn'), eva('lgb_round')

accuracy: 0.6955 	precision: 0.6212697377840808 	recall: 0.5987076558751883 	f1: 0.6027102765173292
accuracy: 0.684 	precision: 0.6162061359268345 	recall: 0.6003210947296718 	f1: 0.6062267975508856
accuracy: 0.6995 	precision: 0.6253411371368436 	recall: 0.6173560306045883 	f1: 0.6186924975842738
accuracy: 0.6345 	precision: 0.554561998241603 	recall: 0.5189400144582846 	f1: 0.5221619533184397
accuracy: 0.4485 	precision: 0.4687899120153495 	recall: 0.40073731519981315 	f1: 0.39445434093244286


(None, None, None, None, None)

In [40]:
df.head()

,review_id,bert1,bert2,bert3,rnn,lgb,cat,all_mean,norm_mean,lgb_round
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,4,4.360081,valid,3,3,4
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,3.824301,valid,5,5,4
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,5,2.929254,valid,4,4,3
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.088829,valid,5,5,3
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,2,2.831013,valid,3,3,3


In [41]:
df['majority_5'] = df.apply(
    lambda x: np.bincount(np.array(
        [x['bert1'], x['bert2'], x['bert3'], x['rnn'], x['lgb_round']])).argmax(), axis=1)
df['majority_5'] = df['majority_5'].apply(lambda x: round_star(x))
df.head()

,review_id,bert1,bert2,bert3,rnn,lgb,cat,all_mean,norm_mean,lgb_round,majority_5
0,ewaMkuxfjfCX3ndtk0oRTw,3,3,3,4,4.360081,valid,3,3,4,3
1,SzkuvIpf0lcVaDgk5x2ipg,5,5,5,5,3.824301,valid,5,5,4,5
2,ZGg6emL3sqhyyqHD_DJEag,5,5,4,5,2.929254,valid,4,4,3,5
3,IXYLmBVvRVFJZEhnhE_YsQ,5,5,5,5,3.088829,valid,5,5,3,5
4,dao7SEXhBuMk7TieM4gEZA,3,3,2,2,2.831013,valid,3,3,3,3


In [42]:
eva('majority_5')

accuracy: 0.699 	precision: 0.6231186669002958 	recall: 0.6076871422079646 	f1: 0.6136311639076075


In [43]:
df.columns

Index(['review_id', 'bert1', 'bert2', 'bert3', 'rnn', 'lgb', 'cat', 'all_mean',
       'norm_mean', 'lgb_round', 'majority_5'],
      dtype='object')

In [47]:
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

X = df[df.cat == 'valid'][['bert1', 'bert2', 'bert3', 'rnn', 'lgb']]
y = y_val

train = lgb.Dataset(
    data=X, 
    label=y)

# tune the parameter with grid search (with cross validation)
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_data_in_leaf': [1, 4, 8, 16]
}

cv_results = []

for hyperparams in tqdm(list(ParameterGrid(param_grid))):
    fixed = {
        "objective": 'regression',
        "learning_rate": 0.01
    }
    hyperparams.update(fixed)

    validation_summary = lgb.cv(
        hyperparams,
        train,
        num_boost_round=6000,
        nfold=5,
        metrics=["rmse"],
        early_stopping_rounds=500,
        verbose_eval=None)

    optimal_num_trees = len(validation_summary["rmse-mean"])
    hyperparams["num_boost_round"] = optimal_num_trees
    cv_results.append((hyperparams, validation_summary["rmse-mean"][-1]))
    
tmp = pd.DataFrame(cv_results)
param = list(tmp.sort_values(by=[1])[0])[0]
train = lgb.Dataset(data=X[:1000], label=y[:1000])
gbm = lgb.train(
    param, 
    train_set=train, 
    num_boost_round=param['num_boost_round'])

100%|██████████| 16/16 [00:48<00:00,  3.61s/it]


In [49]:
acc = accuracy_score(y[1000:], [round_star(s) for s in gbm.predict(X[1000:])])
p, r, f1, _ = precision_recall_fscore_support(y_val, y_val_pre, average="macro")
print("accuracy:", acc, "\tprecision:", p, "\trecall:", r, "\tf1:", f1)

accuracy: 0.675 	precision: 0.6158562692864693 	recall: 0.5986992496273191 	f1: 0.6054109712017525
